In [1]:
from nldi_xstool.XSGen import XSGen
from nldi_xstool.COGridGen import COGridGen
from nldi_xstool.ancillary import queryDEMsShape
import py3dep
from pynhd import NLDI, NHDPlusHR, WaterData


In [2]:
stationid = "09152500"


In [3]:
flw_up = NLDI().navigate_byid(
    fsource="nwissite",
    fid=f"USGS-{stationid}",
    navigation="upstreamMain",
    source="flowlines",
    distance=2)

flw_dwn = NLDI().navigate_byid(
    fsource="nwissite",
    fid=f"USGS-{stationid}",
    navigation="downstreamMain",
    source="flowlines",
    distance=2)

In [4]:
flw_tot =flw_up[::-1].reset_index(drop=True).append(flw_dwn[1:])

In [5]:
flw_tot

,geometry,nhdplus_comid
0,"LINESTRING (-108.45700 38.96766, -108.45561 38...",3232581
1,"LINESTRING (-108.45517 38.97181, -108.45443 38...",3232577
2,"LINESTRING (-108.45446 38.97629, -108.45406 38...",3232573
3,"LINESTRING (-108.45378 38.97856, -108.45365 38...",3232569
1,"LINESTRING (-108.45236 38.98708, -108.45351 38...",3232553


In [6]:
len(flw_tot)

5

In [7]:
flw_tot.to_crs("epsg:3857", inplace=True)
flw_tot.plot()
# catchments.to_crs("epsg:3857", inplace=True)
# catchments.plot()

flw_tot_b = flw_tot.buffer(1000)
minx = min(flw_tot_b.geometry.bounds.minx)
miny = min(flw_tot_b.geometry.bounds.miny)
maxx = max(flw_tot_b.geometry.bounds.maxx)
maxy = max(flw_tot_b.geometry.bounds.maxy)
print(minx, miny, maxx, maxy)

dem = py3dep.get_map( 
    "DEM",
    tuple((minx, miny, maxx, maxy)), 
    resolution=10, 
    geo_crs="epsg:3857", 
    crs="epsg:3857")

grid = COGridGen(
    cl_geom=flw_tot,
    nx=20,
    ny=30,
    width=1000
)

tmp=0

-12075044.650454653 4716039.578198419 -12071716.654702356 4721039.572030521
22


In [8]:
dims = grid.getGridDims()

In [9]:
dims

(20, 30)

In [10]:
values = grid.getXYGrid()

In [19]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.set_aspect('equal')
ax.scatter(values[0], values[1])

plt.show()